<a href="https://colab.research.google.com/github/sharmasgs/sentiment-analysis-imdb/blob/main/IMBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import re
import nltk
from nltk.corpus import  stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [34]:
df=pd.read_csv('/content/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [35]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [36]:
df.isnull().sum()

,0
review,0
sentiment,0


In [37]:
df.duplicated().sum()

np.int64(418)

In [38]:
df.drop_duplicates(inplace=True)

In [39]:
df.duplicated().sum()

np.int64(0)

In [40]:
def tag_removal(raw_text):
  cleaned_text =re.sub('<.*?','',raw_text)
  return(cleaned_text)

In [41]:
df['review'] =df['review'].apply(tag_removal)

In [42]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. br />br />The f...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [43]:
df['review']=df['review'].apply(lambda x:str(x).lower())

In [44]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. br />br />the f...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
sw_list =stopwords.words('english')
df['review'] = df['review'].apply(lambda x:[item for item in x.split()if item not in sw_list]).apply(lambda x:' '.join(x))

In [47]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. br />br />the fil...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
49995,thought movie right good job. creative origina...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,catholic taught parochial elementary schools n...,negative
49998,going disagree previous comment side maltin on...,negative


In [48]:
X = df['review']
y=df['sentiment']

In [49]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. br />br />the fil...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
49995,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [50]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
49995,positive
49996,negative
49997,negative
49998,negative


In [51]:
encoder =LabelEncoder()
y= encoder.fit_transform(y)

In [52]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
print(X_train.shape, X_test.shape)

(39665,) (9917,)


In [55]:
cv=CountVectorizer()


In [56]:
X_train_bow = cv.fit_transform(X_train)
X_test_bow =cv.transform(X_test)

In [57]:
X_train_bow.shape

(39665, 93918)

In [58]:
model = MultinomialNB()
model.fit(X_train_bow,y_train)

MultinomialNB()

In [59]:
y_pred = model.predict(X_test_bow)
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.8600383180397297


In [60]:
tfidf =TfidfVectorizer()


In [61]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [62]:
rf= RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)
y_pred =rf.predict(X_test_tfidf)
accuracy =accuracy_score(y_test,y_pred)